In [1]:
from decouple import config
import requests

In [2]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [3]:
api_key = ALPHA_VANTAGE_API_KEY

params = {
    "api_key": api_key,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY",
}

url = url = (
    "https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=1min&apikey={api_key}".format(
        **params
    )
)

response = requests.get(url)
data = response.json()

In [ ]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

In [5]:
results = data[dataset_key]

In [ ]:
timestamp_str = list(results.keys())[0]
timestamp_str

In [ ]:
result_value = results.get(timestamp_str)
result_value

In [10]:
import pytz
from datetime import datetime
from decimal import Decimal


def transform_alpha_vantage_result(timestamp_str, result):
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(
        datetime.strptime(timestamp_str, timestamp_format)
    ).astimezone(utc)

    return {
        "open_price": Decimal(result["1. open"]),
        "close_price": Decimal(result["4. close"]),
        "high_price": Decimal(result["2. high"]),
        "low_price": Decimal(result["3. low"]),
        "number_of_trades": None,
        "volume": int(result["5. volume"]),
        "volume_weighted_average": None,
    }

In [ ]:
transform_alpha_vantage_result(timestamp_str, result_value)

In [ ]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)